# LangChain for LLM Application Development

ref: https://learn.deeplearning.ai/langchain



## Setup

Python packages needed:
`pip install python-dotenv redlines panel openai langchain`

In [29]:
import os, time
from IPython.display import display, HTML, Markdown, Latex, JSON

from redlines import Redlines

import openai

In [30]:
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())

# openai.api_key  = os.getenv('OPENAI_API_KEY')

In [31]:
openai.api_key = os.environ["OPENAI_API_KEY"]

**helper functions**

To use OpenAI's gpt-3.5-turbo model and the chat completions endpoint, this helper function will make it easier to use prompts and look at the generated outputs.


In [32]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    
    messages = [
        {
            "role": "user", 
            "content": prompt
        }
    ]
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # the degree of randomness of the model's output
    )
    
    return response.choices[0].message["content"]


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, debug=False):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    
    if debug:
        print(f"Returned Message: {str(response.choices[0].message)}" )
    
    return response.choices[0].message["content"]

In [5]:
# this is the so called Pirate English
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [6]:
style = """American English in a calm and respectful tone"""

prompt = f"""Translate the text that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [7]:
response = get_completion(prompt)

response

"I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie. To add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?"

## LangChain Models, Prompts and Parsers

Do the same using LangChain.

### Using `ChatPromptTemplate` to generate Prompt

In [33]:
template_string = """\
Translate the text that is delimited by triple backticks \
into a style that is {style}.
text: ```{text}```
"""

In [34]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [35]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}.\ntext: ```{text}```\n', template_format='f-string', validate_template=True)

In [36]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [37]:
customer_style = """American English in a calm and respectful tone"""

In [38]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [39]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [40]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [41]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone.\ntext: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


### Pick up Model to Call

In [42]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.0) #use temperature to control the randomness and creativity

chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

In [43]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! To add insult to injury, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?


In [44]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [45]:
service_style_pirate = """polite tone that speaks in English Pirate"""

In [46]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is polite tone that speaks in English Pirate.
text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [47]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, me hearty customer! I be sorry to inform ye that the warranty don't be coverin' the expenses o' cleaning yer kitchen. Ye see, it be yer own fault for misusin' yer blender by forgettin' to put the lid on before startin' it. Aye, tough luck! Farewell and may fair winds be at yer back!


### Output Parsers

Start with defining what the LLM output looks like

In [48]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [49]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [50]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [51]:
messages = prompt_template.format_messages(text=customer_review)

chat = ChatOpenAI(temperature=0.0)

response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [52]:
type(response.content)

str

**To parse the LLM output string into a Python dictionary**

In [53]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [56]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? \
    Answer True if yes, False if not or unknown."
)
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? \
    If this  information is not found, output -1."
)
price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value \
    or price, and output them as a comma separated Python list."
)

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [57]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [58]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else?     Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive?     If this  information is not found, output -1.
	"price_value": string  // Extract any sentences about the value     or price, and output them as a comma separated Python list.
}
```


In [59]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else?     Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive?     If this  information is not found, output -1.
	"price_value": string  // Extract any sentences about the value     or price, and output them as a comma separated Python list.
}
```


In [60]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [61]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "

In [62]:
response = chat(messages)

In [63]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [64]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': True,
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

## Memory

- **Write clear and specific instructions**
- **Give the model time and space to “think”**


## Chains

## Question and Answer

## Evaluation


In [43]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Agents
Using  Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

### Translation
ChatGPT is trained with sources in many languages. This gives the model the ability to do translation.